# Basic Web Scraping 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
import requests
page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [3]:
page.status_code

200

In [4]:
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [7]:
list(soup.children)

['html',
 '\n',
 <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [8]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [9]:
html = list(soup.children)[2]

In [10]:
list(html.children)

['\n',
 <head>
 <title>A simple example page</title>
 </head>,
 '\n',
 <body>
 <p>Here is some simple content for this page.</p>
 </body>,
 '\n']

In [11]:
body = list(html.children)[3]

In [12]:
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [13]:
p = list(body.children)[1]

In [14]:
p.get_text()

'Here is some simple content for this page.'

# Weather Extraction

In [15]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Clear, with a low around 44. Northwest wind 5 to 9 mph, with gusts as high as 20 mph. " class="forecast-icon" src="newimages/medium/nskc.png" title="Tonight: Clear, with a low around 44. Northwest wind 5 to 9 mph, with gusts as high as 20 mph. "/>
 </p>
 <p class="short-desc">
  Clear
 </p>
 <p class="temp temp-low">
  Low: 44 °F
 </p>
</div>


In [16]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Tonight
Clear
Low: 44 °F


In [17]:
img = tonight.find("img")
desc = img['title']
print(desc)

Tonight: Clear, with a low around 44. Northwest wind 5 to 9 mph, with gusts as high as 20 mph. 


In [18]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Tonight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight',
 'Tuesday',
 'TuesdayNight']

In [19]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Clear', 'Sunny', 'Clear', 'Sunny', 'Clear', 'Sunny', 'Mostly Clear', 'Mostly Sunny', 'Mostly Clear']
['Low: 44 °F', 'High: 63 °F', 'Low: 46 °F', 'High: 64 °F', 'Low: 45 °F', 'High: 63 °F', 'Low: 45 °F', 'High: 64 °F', 'Low: 46 °F']
['Tonight: Clear, with a low around 44. Northwest wind 5 to 9 mph, with gusts as high as 20 mph. ', 'Saturday: Sunny, with a high near 63. North wind 9 to 14 mph, with gusts as high as 20 mph. ', 'Saturday Night: Clear, with a low around 46. North wind 11 to 16 mph, with gusts as high as 30 mph. ', 'Sunday: Sunny, with a high near 64. North northeast wind 5 to 7 mph. ', 'Sunday Night: Clear, with a low around 45. West wind around 6 mph becoming calm  in the evening. ', 'Monday: Sunny, with a high near 63.', 'Monday Night: Mostly clear, with a low around 45.', 'Tuesday: Mostly sunny, with a high near 64.', 'Tuesday Night: Mostly clear, with a low around 46.']


In [20]:
import pandas as pd
weather = pd.DataFrame({
        "period": periods, 
        "short_desc": short_descs, 
        "temp": temps, 
        "desc":descs
    })
weather

,period,short_desc,temp,desc
0,Tonight,Clear,Low: 44 °F,"Tonight: Clear, with a low around 44. Northwes..."
1,Saturday,Sunny,High: 63 °F,"Saturday: Sunny, with a high near 63. North wi..."
2,SaturdayNight,Clear,Low: 46 °F,"Saturday Night: Clear, with a low around 46. N..."
3,Sunday,Sunny,High: 64 °F,"Sunday: Sunny, with a high near 64. North nort..."
4,SundayNight,Clear,Low: 45 °F,"Sunday Night: Clear, with a low around 45. Wes..."
5,Monday,Sunny,High: 63 °F,"Monday: Sunny, with a high near 63."
6,MondayNight,Mostly Clear,Low: 45 °F,"Monday Night: Mostly clear, with a low around 45."
7,Tuesday,Mostly Sunny,High: 64 °F,"Tuesday: Mostly sunny, with a high near 64."
8,TuesdayNight,Mostly Clear,Low: 46 °F,"Tuesday Night: Mostly clear, with a low around..."


In [21]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    44
1    63
2    46
3    64
4    45
5    63
6    45
7    64
8    46
Name: temp_num, dtype: object

In [22]:
weather["temp_num"].mean()

53.333333333333336